# CPE342 - Karena Task1 V7


In [1]:
import pandas as pd
import numpy as np
import optuna
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn & Metrics
from sklearn.metrics import fbeta_score, roc_auc_score, make_scorer
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier, IsolationForest # เพิ่ม IsolationForest
from sklearn.pipeline import Pipeline # เปลี่ยนจาก imblearn pipeline เป็น sklearn pipeline ปกติ

# Gradient Boosting Libraries
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

warnings.filterwarnings('ignore')

# Global Config
RANDOM_STATE = 42
N_FOLDS = 5
TARGET_COL = "is_cheater"

# Features List (Based on your Datacard)
DATACARD_FEATURES = [
    "kill_death_ratio", "headshot_percentage", "win_rate", "accuracy_score", 
    "kill_consistency", "reaction_time_ms", "account_age_days", "level", 
    "level_progression_speed", "friend_network_size", "reports_received", 
    "device_changes_count", "input_consistency_score", "weapon_switch_speed", 
    "movement_pattern_score", "aiming_smoothness", "spray_control_score", 
    "game_sense_score", "communication_rate", "team_play_score", 
    "buy_decision_score", "map_knowledge", "clutch_success_rate", 
    "first_blood_rate", "survival_time_avg", "damage_per_round", 
    "utility_usage_rate", "crosshair_placement"
]

# Columns ที่ข้อมูลมีการกระจายตัวแบบเบ้ (Skewed) ที่เราจะทำ Log Transform
LOG_COLS = ['survival_time_avg', 'damage_per_round', 'account_age_days', 'reaction_time_ms']

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data & Clean Target

In [2]:
# 1. Load Data (แก้ Path ตามเครื่องของคุณ)
train_df = pd.read_csv("C:/Users/DELL/Desktop/CPE342_Karena/Dataset/task1/train.csv")
test_df = pd.read_csv("C:/Users/DELL/Desktop/CPE342_Karena/Dataset/task1/test.csv")

# 2. Clean Target Variable
# ลบแถวที่ไม่มีค่า Target (is_cheater) ทิ้ง
initial_len = len(train_df)
train_df = train_df.dropna(subset=[TARGET_COL])
train_df.reset_index(drop=True, inplace=True)

print(f"Original rows: {initial_len}")
print(f"Rows after dropping missing target: {len(train_df)}")

Original rows: 99872
Rows after dropping missing target: 97748


## Advanced Feature Engineering

In [3]:
def preprocess_and_engineer_v2(df):
    # 1. เลือกเฉพาะ Column ที่มีใน Datacard
    available_feats = [c for c in DATACARD_FEATURES if c in df.columns]
    df_clean = df[available_feats].copy()
    
    # 2. เติมค่า Missing (Median)
    imputer = SimpleImputer(strategy='median')
    df_filled = pd.DataFrame(imputer.fit_transform(df_clean), columns=df_clean.columns, index=df_clean.index)
    
    # --- UPGRADE 1: Log Transform ---
    # ช่วยให้โมเดลรับมือกับ Outlier ในค่าตัวเลขสูงๆ ได้ดีขึ้น
    for col in LOG_COLS:
        if col in df_filled.columns:
            df_filled[f'log_{col}'] = np.log1p(df_filled[col])
            # (Option) ถ้าอยากเก็บค่าเดิมไว้ก็ไม่ต้อง drop แต่ถ้าจะลด feature ซ้ำซ้อนให้ drop ค่าดิบทิ้ง
            # df_filled.drop(columns=[col], inplace=True) 

    # --- UPGRADE 2: Context-Aware Features ---
    
    # 2.1 Reports Efficiency: คนเก่งฆ่าเยอะรีพอร์ตเยอะเป็นเรื่องปกติ แต่คนโกงฆ่าน้อยอาจโดนรีพอร์ตหนัก
    df_filled['report_per_kill'] = df_filled['reports_received'] / (df_filled['kill_death_ratio'] + 1e-5)
    
    # 2.2 Suspicion Score: ไอดีใหม่ + ยิงหัวแม่น + ชนะบ่อย = โกงแน่ๆ
    df_filled['sus_score'] = (df_filled['headshot_percentage'] * df_filled['win_rate']) / (df_filled['account_age_days'] + 1e-5)

    # --- Original Engineered Features (เก็บตัวที่ดีไว้) ---
    df_filled['skill_consistency'] = df_filled['game_sense_score'] / (df_filled['accuracy_score'] + 1e-5)
    df_filled['suspicious_aim'] = df_filled['headshot_percentage'] * df_filled['reaction_time_ms']
    df_filled['noob_movement_with_more_kills'] = df_filled['kill_death_ratio'] / (df_filled['movement_pattern_score'] + 1e-5)
    df_filled['Aim_to_Brain_Ratio'] = (df_filled['accuracy_score'] + df_filled['headshot_percentage']) /(df_filled['map_knowledge'] + df_filled['utility_usage_rate'] + 1e-5)
    df_filled['Sus_Headshot_Rate'] = (df_filled['headshot_percentage']) / (df_filled['crosshair_placement'] + 1e-5)

    # --- Clean Noise ---
    # ลบ Feature ที่เราวิเคราะห์แล้วว่า Correlation ต่ำมาก หรือเป็น Noise
    noise_cols = ['friend_network_size', 'device_changes_count', 'weapon_switch_speed', 'clutch_success_rate', 'buy_decision_score']
    df_filled = df_filled.drop(columns=[c for c in noise_cols if c in df_filled.columns], errors='ignore')

    return df_filled

print("Feature Engineering function updated.")

Feature Engineering function updated.


## Apply Engineering & Add Isolation Forest

In [4]:
# 1. Apply Feature Engineering
print("Applying Feature Engineering...")
X = preprocess_and_engineer_v2(train_df)
y = train_df[TARGET_COL]
X_test = preprocess_and_engineer_v2(test_df)

# 2. --- UPGRADE 3: Anomaly Detection Feature ---
print("Training Isolation Forest to detect anomalies...")

# contamination=0.05 หมายถึงเราคาดเดาว่ามี Outlier (คนโกง/คนแปลก) ประมาณ 5%
iso = IsolationForest(n_estimators=100, contamination=0.05, random_state=RANDOM_STATE, n_jobs=-1)

# Fit กับ Train Set
iso.fit(X)

# สร้าง Feature ใหม่: 'anomaly_score' (ยิ่งติดลบ ยิ่งแปลก)
X['anomaly_score'] = iso.decision_function(X)
X_test['anomaly_score'] = iso.decision_function(X_test)

print(f"Data Shape with Anomaly Score: {X.shape}")

Applying Feature Engineering...
Training Isolation Forest to detect anomalies...
Data Shape with Anomaly Score: (97748, 35)


## Correlation Analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

# --- Correlation Analysis Section ---

# 1. เตรียมข้อมูล: นำ Features (X) และ Target (y) มารวมกันเพื่อวิเคราะห์
df = X.copy()
df[TARGET_COL] = y

# 2. กำหนดค่าพารามิเตอร์
CORR_METHOD = "pearson"          # "pearson" or "spearman" (or "kendall")

# 3. แยก Numeric Columns
numeric_df = df.select_dtypes(include=[np.number]).copy()
# ตรวจสอบว่ามี Target อยู่ใน numeric_df หรือไม่ ถ้าไม่มีให้ดึงมาจาก df
if TARGET_COL not in numeric_df.columns:
    numeric_df[TARGET_COL] = df[TARGET_COL]

# 4. จัดการกับ Non-Numeric Columns (ถ้ามี)
non_num_cols = [c for c in df.columns if c not in numeric_df.columns and c != TARGET_COL]
for c in non_num_cols:
    try:
        le = LabelEncoder()
        numeric_df[c] = le.fit_transform(df[c].astype(str))
    except Exception:
        # if encoding fails, skip the column
        print(f"Warning: skipping column {c} from encoding")

# 5. Loop สร้าง Heatmap แยกตาม Class ของ Target
unique_targets = sorted(df[TARGET_COL].unique())
print("Found unique target values:", unique_targets)

for cls in unique_targets:
    # สร้าง Binary Target สำหรับ Class นั้นๆ (One-vs-Rest)
    bin_name = f"target_is_{cls}"
    numeric_df[bin_name] = (df[TARGET_COL] == cls).astype(int)

    # คำนวณ Correlation Matrix
    corr = numeric_df.corr(method=CORR_METHOD)

    # จัดลำดับ Columns ให้ target_is_X ไปอยู่ขวาสุดเพื่อให้ดูง่าย
    cols = [c for c in corr.columns if c != bin_name and c != TARGET_COL] + [bin_name]
    # ตัดเอาเฉพาะ Columns ที่เราสนใจ (เอา Target เดิมออก เพื่อดูเทียบกับ Binary Target ใหม่)
    corr = corr.loc[cols, cols]

    # Plot heatmap
    plt.figure(figsize=(14, 12)) # ปรับขนาดให้ใหญ่ขึ้นเล็กน้อยเพื่อให้เห็น Label ชัดเจน
    ax = sns.heatmap(
        corr,
        annot=False, # ปิดตัวเลขในช่องเพราะ Feature เยอะอาจจะลายตา (เปิด True ถ้าอยากเห็นเลข)
        cmap='coolwarm',
        linewidths=0.3,
        center=0
    )
    ax.set_title(f"Correlation heatmap (target = {cls}) [{CORR_METHOD}]")
    plt.tight_layout()
    plt.show()

    # แบบใหม่ (ดูทิศทาง: บวก=ไปทาง Class นั้น, ลบ=ตรงข้าม)
    # แสดง Top 20 features ที่มีความสัมพันธ์ทางบวก (บ่งชี้ว่าเป็น Class นี้)
    corr_with_target = corr[bin_name].drop(bin_name).sort_values(ascending=False)
    print(f"\nTop 20 features correlated with target == {cls} (Positive correlation implies trait of this class):")
    print(corr_with_target.head(20))

    # ลบคอลัมน์ Binary ทิ้งเพื่อเตรียม Loop รอบถัดไป
    numeric_df.drop(columns=[bin_name], inplace=True)

## Calculate Class Weights

In [5]:
# นับจำนวน Class
num_neg = (y == 0).sum()
num_pos = (y == 1).sum()

# สูตรคำนวณ: จำนวนคนปกติ / จำนวนคนโกง
scale_pos_weight = num_neg / num_pos

print(f"Class 0 (Normal): {num_neg}")
print(f"Class 1 (Cheater): {num_pos}")
print(f"Calculated scale_pos_weight for XGBoost: {scale_pos_weight:.4f}")

Class 0 (Normal): 63619
Class 1 (Cheater): 34129
Calculated scale_pos_weight for XGBoost: 1.8641


## Optuna Tuning

### XGBoost

In [6]:
def objective_xgb_weighted(trial):
    params = {
        # Hyperparameters
        'n_estimators': trial.suggest_int('n_estimators', 300, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        
        # --- Key Config for Imbalanced Data (NO SMOTE) ---
        'scale_pos_weight': scale_pos_weight,  # ใส่ Weight ที่คำนวณไว้
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'random_state': RANDOM_STATE,
        'n_jobs': -1,
        'verbosity': 0
    }
    
    # Pipeline แบบธรรมดา (Simple & Clean)
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', XGBClassifier(**params))
    ])
    
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    
    # ใช้ scoring='roc_auc' เพื่อวัดผล
    scores = cross_val_score(pipeline, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)
    
    return scores.mean()

print("--- Tuning XGBoost with Class Weights (No SMOTE) ---")
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb_weighted, n_trials=20) # รัน 20 รอบ (เพิ่มได้ถ้ามีเวลา)

print(f"Best XGB AUC: {study_xgb.best_value:.4f}")
best_xgb_params = study_xgb.best_params

# อัปเดต params คงที่กลับเข้าไป
best_xgb_params.update({
    'scale_pos_weight': scale_pos_weight,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'verbosity': 0
})

[I 2025-11-23 12:34:56,809] A new study created in memory with name: no-name-211bdb0c-5086-4bfc-903b-847fe02293f1


--- Tuning XGBoost with Class Weights (No SMOTE) ---


[I 2025-11-23 12:35:04,066] Trial 0 finished with value: 0.8520897537142993 and parameters: {'n_estimators': 335, 'max_depth': 9, 'learning_rate': 0.018057312079281643, 'subsample': 0.7000778443995239, 'colsample_bytree': 0.7994198932291424, 'gamma': 3.4410812141905502, 'min_child_weight': 9}. Best is trial 0 with value: 0.8520897537142993.
[I 2025-11-23 12:35:19,069] Trial 1 finished with value: 0.851172099597893 and parameters: {'n_estimators': 761, 'max_depth': 5, 'learning_rate': 0.06210108979742987, 'subsample': 0.9632410703009366, 'colsample_bytree': 0.9861594151463973, 'gamma': 2.4450962174639717, 'min_child_weight': 7}. Best is trial 0 with value: 0.8520897537142993.
[I 2025-11-23 12:35:31,703] Trial 2 finished with value: 0.8479439219460515 and parameters: {'n_estimators': 625, 'max_depth': 7, 'learning_rate': 0.04458293304560393, 'subsample': 0.602069042676527, 'colsample_bytree': 0.7494632221583096, 'gamma': 3.6817654382038363, 'min_child_weight': 4}. Best is trial 0 with va

Best XGB AUC: 0.8531


### LightGBM

In [7]:
def objective_lgbm_weighted(trial):
    params = {
        # Hyperparameters
        'n_estimators': trial.suggest_int('n_estimators', 400, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        
        # --- Config for Imbalanced Data (NO SMOTE) ---
        'class_weight': 'balanced', # ให้ LGBM จัดการ weight เองอัตโนมัติ
        'objective': 'binary',
        'metric': 'auc',
        'random_state': RANDOM_STATE,
        'n_jobs': -1,
        'verbosity': -1
    }
    
    # Pipeline ธรรมดา (ไม่ต้องมี SMOTE)
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LGBMClassifier(**params))
    ])
    
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_val_score(pipeline, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)
    
    return scores.mean()

print("--- Tuning LightGBM with Class Weights (No SMOTE) ---")
study_lgbm = optuna.create_study(direction='maximize')
study_lgbm.optimize(objective_lgbm_weighted, n_trials=20) 

print(f"Best LGBM AUC: {study_lgbm.best_value:.4f}")
best_lgbm_params = study_lgbm.best_params

# Update constant params
best_lgbm_params.update({
    'class_weight': 'balanced',
    'objective': 'binary',
    'metric': 'auc',
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'verbose': -1
})

[I 2025-11-23 12:40:23,362] A new study created in memory with name: no-name-2e9c0eea-9cbe-4a37-bfb6-4067a836ef2f


--- Tuning LightGBM with Class Weights (No SMOTE) ---


[I 2025-11-23 12:40:26,103] Trial 0 finished with value: 0.8531336520907034 and parameters: {'n_estimators': 911, 'learning_rate': 0.043805171514443936, 'num_leaves': 50, 'max_depth': 3, 'subsample': 0.8530235207489518, 'colsample_bytree': 0.6126932654857209, 'reg_alpha': 9.180231733788624, 'reg_lambda': 1.271471572245999, 'min_child_samples': 25}. Best is trial 0 with value: 0.8531336520907034.
[I 2025-11-23 12:40:29,767] Trial 1 finished with value: 0.8540383094468625 and parameters: {'n_estimators': 741, 'learning_rate': 0.03104478324526414, 'num_leaves': 71, 'max_depth': 5, 'subsample': 0.7247355017652334, 'colsample_bytree': 0.7417839068062162, 'reg_alpha': 7.219998248637134, 'reg_lambda': 4.119594473324582, 'min_child_samples': 72}. Best is trial 1 with value: 0.8540383094468625.
[I 2025-11-23 12:40:35,209] Trial 2 finished with value: 0.8508706315782767 and parameters: {'n_estimators': 490, 'learning_rate': 0.06624646150681625, 'num_leaves': 74, 'max_depth': 7, 'subsample': 0.68

Best LGBM AUC: 0.8540


### CatBoost

In [8]:
def objective_cat_weighted(trial):
    params = {
        # Hyperparameters
        'iterations': trial.suggest_int('iterations', 500, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0),
        
        # --- Config for Imbalanced Data (NO SMOTE) ---
        'auto_class_weights': 'Balanced', # CatBoost จัดการเอง (ดีมาก)
        'loss_function': 'Logloss', 
        'eval_metric': 'AUC',
        'random_state': RANDOM_STATE,
        'verbose': 0,
        'bootstrap_type': 'Bernoulli'
    }
    
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', CatBoostClassifier(**params))
    ])
    
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_val_score(pipeline, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)
    
    return scores.mean()

print("--- Tuning CatBoost with Class Weights (No SMOTE) ---")
study_cat = optuna.create_study(direction='maximize')
study_cat.optimize(objective_cat_weighted, n_trials=15) # CatBoost ช้าหน่อย ลดรอบลงได้

print(f"Best CatBoost AUC: {study_cat.best_value:.4f}")
best_cat_params = study_cat.best_params

# Update constant params
best_cat_params.update({
    'auto_class_weights': 'Balanced',
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'random_state': RANDOM_STATE,
    'verbose': 0,
    'bootstrap_type': 'Bernoulli'
})

[I 2025-11-23 12:43:31,739] A new study created in memory with name: no-name-4553701c-0e1d-4a80-b827-61f775260fec


--- Tuning CatBoost with Class Weights (No SMOTE) ---


[I 2025-11-23 12:43:55,612] Trial 0 finished with value: 0.8539441746186921 and parameters: {'iterations': 979, 'learning_rate': 0.045213802122371326, 'depth': 4, 'l2_leaf_reg': 2.6502622363352857, 'subsample': 0.8979574675102726, 'colsample_bylevel': 0.9309948500384374}. Best is trial 0 with value: 0.8539441746186921.
[I 2025-11-23 12:44:14,062] Trial 1 finished with value: 0.8530401779344018 and parameters: {'iterations': 601, 'learning_rate': 0.05279403732508587, 'depth': 7, 'l2_leaf_reg': 5.88977845512999, 'subsample': 0.8507495628753396, 'colsample_bylevel': 0.7770917390978541}. Best is trial 0 with value: 0.8539441746186921.
[I 2025-11-23 12:44:33,298] Trial 2 finished with value: 0.8526194113865252 and parameters: {'iterations': 1157, 'learning_rate': 0.07582946615686256, 'depth': 4, 'l2_leaf_reg': 3.778235475208253, 'subsample': 0.6504786890623648, 'colsample_bylevel': 0.780424992328009}. Best is trial 0 with value: 0.8539441746186921.
[I 2025-11-23 12:45:04,101] Trial 3 finish

Best CatBoost AUC: 0.8543


## Stacking Ensemble

In [9]:
# Helper Function สร้าง Pipeline
def make_pipeline(classifier):
    return Pipeline([
        ('scaler', StandardScaler()),
        ('clf', classifier)
    ])

# รวม Model ที่จูนมาแล้ว (Best Params)
estimators = [
    ('xgb', make_pipeline(XGBClassifier(**best_xgb_params))),
    ('lgbm', make_pipeline(LGBMClassifier(**best_lgbm_params))),
    ('cat', make_pipeline(CatBoostClassifier(**best_cat_params)))
]

# Meta Learner (ตัวตัดสินใจขั้นสุดท้าย)
blender = LogisticRegression(random_state=RANDOM_STATE, solver='liblinear')

# Stacking Construction
stack_model = StackingClassifier(
    estimators=estimators,
    final_estimator=blender,
    stack_method='predict_proba',
    cv=N_FOLDS,
    n_jobs=-1
)

print("1. Fitting Weighted Stacking Model (Full Ensemble)...")
stack_model.fit(X, y)

print("2. Predicting Test Set...")
# ทำนายผลเป็น Probability เก็บไว้
y_test_proba = stack_model.predict_proba(X_test)[:, 1]

print("Done.")

1. Fitting Weighted Stacking Model (Full Ensemble)...
2. Predicting Test Set...
Done.


## Threshold Optimization & Submission

In [10]:
print("3. Finding Best Threshold for F2 Score...")

# ใช้ Cross Validation Predict เพื่อหา Threshold จากข้อมูล Train
y_train_cv_proba = cross_val_predict(stack_model, X, y, cv=N_FOLDS, method='predict_proba', n_jobs=-1)[:, 1]

thresholds = np.arange(0.10, 0.90, 0.005)
f2_scores = []

for thresh in thresholds:
    y_pred_temp = (y_train_cv_proba >= thresh).astype(int)
    score = fbeta_score(y, y_pred_temp, beta=2)
    f2_scores.append(score)

best_idx = np.argmax(f2_scores)
best_thresh = thresholds[best_idx]
best_f2 = f2_scores[best_idx]

print(f"Optimal Threshold found: {best_thresh:.4f}")
print(f"Max F2 Score: {best_f2:.4f}")

# สร้าง Final Prediction สำหรับ Test Set
y_test_final = (y_test_proba >= best_thresh).astype(int)

3. Finding Best Threshold for F2 Score...
Optimal Threshold found: 0.1050
Max F2 Score: 0.8160


In [11]:
submission_df = pd.read_csv("final_submission.csv") # ตรวจสอบชื่อไฟล์เดิมของคุณ
submission_df['task1'] = y_test_final
submission_df.to_csv("final_submission_v7_optimized.csv", index=False)
print("Submission saved to 'final_submission_v7_optimized.csv'")
print(submission_df.head(20))

Submission saved to 'final_submission_v7_optimized.csv'
          id  task1  task2          task3  task4  task5
0   ANS00001      1      2     517.156944      1      0
1   ANS00002      0      0    1413.611996      3      0
2   ANS00003      1      0  158344.753397      3      1
3   ANS00004      0      0     117.963622      0      0
4   ANS00005      1      0     101.542132      3      0
5   ANS00006      1      2     609.682928      2      0
6   ANS00007      1      1       0.000000      1      0
7   ANS00008      1      0   17357.013216      3      0
8   ANS00009      1      0       0.000000      0      0
9   ANS00010      0      1       0.000000      3      0
10  ANS00011      0      2     128.685354      1      0
11  ANS00012      0      0     432.884720      3      0
12  ANS00013      0      0      25.834546      1      0
13  ANS00014      0      1    1670.033716      2      0
14  ANS00015      1      2       0.000000      4      0
15  ANS00016      0      0       0.000000      0